# Chapter 4. Estimating Proportions
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap04.html)

A reminder of Bayes’s Theorem:

$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$

or

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
# none for now

In [ ]:
include("pmf.jl")
import .ProbabilityMassFunction as Pmf


## Functionality developed in this chapter

In [ ]:
# none for now

## The Euro Problem

In Information Theory, Inference, and Learning Algorithms, David MacKay poses this problem:

“A statistical statement appeared in The Guardian on Friday January 4, 2002:

> When spun on edge 250 times, a Belgian one-euro coin came up heads 140 times
> and tails 110. "It looks very suspicious to me", said Barry Blight, a statistics
> lecturer at the London School of Economics. "If the coin were unbiased, the
> chance of getting a result as extreme as that would be less than 7%."

“But [MacKay asks] do these data give evidence that the coin is biased rather than fair?”


